In [83]:
from flask import Flask, render_template, request, json
#from flaskext.mysql import MySQL


app = Flask(__name__)


RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request.  Consult the documentation on testing for
information about how to avoid this problem.

NameError: name 'MySQL' is not defined

In [43]:
@app.route('/showSignUp')
def showSignUp():
    return render_template('sign-up.html')

@app.route('/signUp',methods=['POST'])
def signUp():
 
    # read the posted values from the UI
    _name = request.form['inputName']
    _email = request.form['inputEmail']
    _password = request.form['inputPassword']
 
    # validate the received values
    if _name and _email and _password:
        return json.dumps({'html':'<span>All fields good !!</span>'})
    else:
        return json.dumps({'html':'<span>Enter the required fields</span>'})
    
@app.route("/")
def main():
    return render_template('index.html')

In [44]:
if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Apr/2017 16:05:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2017 16:05:32] "GET /showSignUp HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2017 16:05:32] "GET /static/signup.css HTTP/1.1" 404 -
127.0.0.1 - - [16/Apr/2017 16:05:33] "GET /static/signup.css HTTP/1.1" 404 -
127.0.0.1 - - [16/Apr/2017 16:05:37] "POST /signUp HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2017 16:05:39] "POST /signUp HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2017 16:05:45] "GET /static/signup.css HTTP/1.1" 404 -
